In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="XyR1pKoVYN1NNNzSC0iA")
project = rf.workspace("deteccion-de-residuos-modulo-4").project("reciclaje-c9ri0-jtmne")
version = project.version(3)
dataset = version.download("yolov11")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Reciclaje-3 in yolov11:: 100%|██████████| 468/468 [00:00<00:00, 7328.13it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os

folder_path = "/content/Reciclaje-3"
new_folder_path = "/content/yolov11"

if os.path.exists(new_folder_path):
    print("La carpeta ya se llama 'yolov11'.")
else:
    os.rename(folder_path, new_folder_path)
    print("Carpeta renombrada correctamente a 'yolov11'.")

Carpeta renombrada correctamente a 'yolov11'.


### División del dataset (Entrenamiento 70 - Evaluacion 20 - Prueba 10)

In [3]:
import os

# Contar imágenes y calcular porcentaje
train_images = len(os.listdir("yolov11/train/images"))
val_images = len(os.listdir("yolov11/valid/images"))
test_images = len(os.listdir("yolov11/test/images"))
total_images = train_images + val_images + test_images

# Mostrar los resultados con porcentaje entero
for set_name, images in [("Entrenamiento", train_images), ("Validación", val_images), ("Prueba", test_images)]:
    print(f"Imágenes en {set_name}: {images} ({round((images / total_images) * 100)}%)")


Imágenes en Entrenamiento: 160 (70%)
Imágenes en Validación: 45 (20%)
Imágenes en Prueba: 23 (10%)


<h1>Aumento de datos</h1>
(no ejecutar, nuevamente ya se ejecuto y se genero el aumento de datos en las carpetas correspondientes de entrenamiento, evaluacion y prueba, en caso de ejecutarlo borrar el la carpeta yolov11, ejecutando en el codigo !rm -r yolov11 y correrlo nuevamente
para volver a generar el conjunto de datos )

In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [23]:
#!rm -r runs/detect/train5

In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# Directorios de las imágenes
train_dir = 'yolov11/train/images'
valid_dir = 'yolov11/valid/images'
test_dir = 'yolov11/test/images'

# Asegurarse de que las carpetas existan
for dir_path in [train_dir, valid_dir, test_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Inicializar generador de aumento de imágenes con opciones variadas
datagen = ImageDataGenerator(
    rotation_range=40,  # Rotación aleatoria entre -40 y +40 grados
    width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
    height_shift_range=0.2,  # Desplazamiento vertical aleatorio
    shear_range=0.2,  # Distorsión aleatoria en los ángulos
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Flip horizontal aleatorio
    vertical_flip=True,  # Flip vertical aleatorio
    fill_mode='nearest'  # Relleno para las áreas creadas por transformaciones
)

# Función para generar imágenes aumentadas solo si no existen
def generate_unique_image(src_image, src_image_name, target_dir, num_augmentations=4):
    counter = 0
    generated_files = []

    for _ in datagen.flow(src_image, batch_size=1, save_to_dir=target_dir,
                          save_prefix=f"{src_image_name.split('.')[0]}", save_format='jpg'):
        new_image_name = f"{src_image_name.split('.')[0]}_{counter}.jpg"

        # Verificar si la imagen generada ya existe en la carpeta destino
        if os.path.exists(os.path.join(target_dir, new_image_name)):
            print(f"Imagen ya creada: {new_image_name}. No se generará nuevamente.")
            continue  # Si existe, no la generamos nuevamente

        # Si no existe, añadimos el nombre de la imagen generada y la generamos
        generated_files.append(new_image_name)
        counter += 1

        if counter >= num_augmentations:  # Limitar el número de imágenes generadas
            break

# Generar aumentos para cada imagen en las carpetas train, valid y test
for folder in [train_dir, valid_dir, test_dir]:
    for image_name in os.listdir(folder):
        if image_name.endswith(('.jpg', '.png')):  # Asegurarse de que sea una imagen
            # Cargar imagen
            img = load_img(os.path.join(folder, image_name))
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # Generar aumentos (por ejemplo, x10)
            generate_unique_image(x, image_name, folder, num_augmentations=10)

print("Proceso de aumento de imágenes completado.")


Proceso de aumento de imágenes completado.


In [7]:
# verificamos la cantidad de aumentacion
import os

# Contar imágenes y calcular porcentaje
train_images = len(os.listdir("yolov11/train/images"))
val_images = len(os.listdir("yolov11/valid/images"))
test_images = len(os.listdir("yolov11/test/images"))
total_images = train_images + val_images + test_images

# Mostrar los resultados con porcentaje entero
for set_name, images in [("Entrenamiento", train_images), ("Validación", val_images), ("Prueba", test_images)]:
    print(f"Imagenes en {set_name}: {images} ({round((images / total_images) * 100)}%)")

Imagenes en Entrenamiento: 1760 (70%)
Imagenes en Validación: 495 (20%)
Imagenes en Prueba: 253 (10%)


### Entrenamiento del modelo

Ya que tienes tus imágenes aumentadas y organizadas en directorios de entrenamiento, validación y prueba, entrenaremos el modelo con YOLOv11 y TensorFlow



*dependencias*

In [8]:
# Instalar las dependencias necesarias
!pip install ultralytics
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 23.1 MB/s eta 0:00:00


In [9]:
with open('yolov11/data.yaml', 'r') as f:
    print(f.read())

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 4
names: ['Metal', 'Papel', 'Plastico', 'Vidrio']

roboflow:
  workspace: deteccion-de-residuos-modulo-4
  project: reciclaje-c9ri0-jtmne
  version: 3
  license: Public Domain
  url: https://universe.roboflow.com/deteccion-de-residuos-modulo-4/reciclaje-c9ri0-jtmne/dataset/3


In [10]:
!pip install torch --upgrade


In [11]:
import torch
print(torch.cuda.is_available())


True


In [12]:
!nvidia-smi


Sun Jan 12 16:55:02 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import os
import shutil

# 1. Primero, creamos el directorio necesario
os.makedirs('/content/datasets/yolov11/valid/images', exist_ok=True)
os.makedirs('/content/datasets/yolov11/train/images', exist_ok=True)
os.makedirs('/content/datasets/yolov11/test/images', exist_ok=True)

# 2. Copiamos las imágenes a la nueva ubicación
# Asumiendo que las imágenes están en tu estructura actual
shutil.copytree('yolov11/valid', '/content/datasets/yolov11/valid', dirs_exist_ok=True)
shutil.copytree('yolov11/train', '/content/datasets/yolov11/train', dirs_exist_ok=True)
shutil.copytree('yolov11/test', '/content/datasets/yolov11/test', dirs_exist_ok=True)

# 3. Modificamos el archivo data.yaml
yaml_content = f"""
path: /content/datasets/yolov11  # ruta base absoluta
train: train/images
val: valid/images
test: test/images
nc: 4
names: ['Metal', 'Papel', 'Plastico', 'Vidrio']
roboflow:
  workspace: deteccion-de-residuos-modulo-4
  project: reciclaje-c9ri0-jtmne
  version: 3
  license: Public Domain
  url: https://universe.roboflow.com/deteccion-de-residuos-modulo-4/reciclaje-c9ri0-jtmne/dataset/3
"""

with open('/content/datasets/yolov11/data.yaml', 'w') as f:
    f.write(yaml_content)

# 4. Verificamos la estructura
print("Verificando estructura de directorios:")
for root, dirs, files in os.walk('/content/datasets/yolov11'):
    print(f"\nDirectorio: {root}")
    if files:
        print(f"Número de archivos: {len(files)}")

# 5. Entrenamos el modelo con la nueva ruta
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(
    data='/content/datasets/yolov11/data.yaml',
    epochs=50,
    imgsz=640,
    batch=8,
    patience=30,
    save=True,
    device='cuda'
)


Verificando estructura de directorios:

Directorio: /content/datasets/yolov11
Número de archivos: 1

Directorio: /content/datasets/yolov11/valid

Directorio: /content/datasets/yolov11/valid/images
Número de archivos: 495

Directorio: /content/datasets/yolov11/valid/labels
Número de archivos: 45

Directorio: /content/datasets/yolov11/test

Directorio: /content/datasets/yolov11/test/images
Número de archivos: 253

Directorio: /content/datasets/yolov11/test/labels
Número de archivos: 23

Directorio: /content/datasets/yolov11/train

Directorio: /content/datasets/yolov11/train/images
Número de archivos: 1760

Directorio: /content/datasets/yolov11/train/labels
Número de archivos: 160
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/datasets/yolov11/data.yaml, epochs=50, time=None, patience=30, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 755k/755k [00:00<00:00, 20.2MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50      1.18G     0.8786      2.334      1.101          2        640: 100%|██████████| 220/220 [00:39<00:00,  5.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:02<00:00, 10.37it/s]

                   all        495        116      0.831     0.0857      0.118     0.0585



50 epochs completed in 0.631 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:03<00:00,  8.08it/s]


                   all        495        116      0.843        0.1      0.118     0.0628
                 Metal         21         61          1          0     0.0859     0.0537
                 Papel          3          4          1          0     0.0854     0.0342
              Plastico          7         16          1          0    0.00694    0.00391
                Vidrio         19         35      0.371        0.4      0.293       0.16
Speed: 0.4ms preprocess, 2.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train


### Ahora podemos proceder con la evaluación y prueba del modelo.

*Evaluacion y Prueba del modelo con los datos de validacion*
<li> Esta generado dentro de runs/detected/train2, desglosando se encuentran los resultados del conjunto de datos de evaluacion con cada una de las metricas de manera grafica.

In [24]:
# Evaluar el modelo con el conjunto de validación
metrics = model.val(
    data="/content/datasets/yolov11/data.yaml",  # Archivo de configuración
    conf=0.1,                                  # Umbral de confianza
    split="val",                                # Usar datos de validación
)

# Imprimir todas las métricas disponibles
print("Todas las métricas disponibles:")
print(metrics.results_dict)  # SIN paréntesis

# Extraer métricas específicas
metrics_dict = metrics.results_dict  # SIN paréntesis
precision = metrics_dict.get("precision", 0)
recall = metrics_dict.get("recall", 0)
mAP50 = metrics_dict.get("mAP_50", 0)
mAP50_95 = metrics_dict.get("mAP_50_95", 0)



Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/datasets/yolov11/valid/labels.cache... 45 images, 450 backgrounds, 0 corrupt: 100%|██████████| 495/495 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 77, len(boxes) = 116. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:04<00:00, 14.38it/s]


                   all        495        116      0.107     0.0857     0.0957     0.0553
                 Metal         21         61          0          0          0          0
                 Papel          3          4          0          0          0          0
              Plastico          7         16          0          0          0          0
                Vidrio         19         35      0.429      0.343      0.383      0.221
Speed: 0.5ms preprocess, 5.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train2
Todas las métricas disponibles:
{'metrics/precision(B)': 0.10714285714285714, 'metrics/recall(B)': 0.08571428571428572, 'metrics/mAP50(B)': 0.09571733158146202, 'metrics/mAP50-95(B)': 0.055302208924703854, 'fitness': 0.05934372119037967}


*Prueba del modelo con los datos de test*

In [25]:
# prueba  del modelo con el conjunto de test
metrics_test = model.val(
    data="/content/datasets/yolov11/data.yaml",  # Archivo de configuración
    conf=0.1,                                  # Umbral de confianza
    split="test",                               # Usar datos de test
)

# Extraer métricas de test
metrics_test_dict = metrics_test.results_dict  # SIN paréntesis
precision_test = metrics_test_dict.get("precision", 0)
recall_test = metrics_test_dict.get("recall", 0)
mAP50_test = metrics_test_dict.get("mAP_50", 0)
mAP50_95_test = metrics_test_dict.get("mAP_50_95", 0)

Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/datasets/yolov11/test/labels.cache... 23 images, 230 backgrounds, 0 corrupt: 100%|██████████| 253/253 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 88, len(boxes) = 107. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  7.91it/s]


                   all        253        107      0.133     0.0513      0.104     0.0657
                 Metal          9         27          0          0          0          0
                 Papel          1         13          0          0          0          0
              Plastico          6         28          0          0          0          0
                Vidrio         11         39      0.533      0.205      0.417      0.263
Speed: 0.9ms preprocess, 8.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train3


In [26]:
import os

# Verificar estructura y número de archivos
print("Verificando estructura del dataset:")
for split in ['train', 'valid', 'test']:
    for subdir in ['images', 'labels']:
        path = f'/content/datasets/yolov11/{split}/{subdir}'
        if os.path.exists(path):
            files = os.listdir(path)
            print(f"{split}/{subdir}: {len(files)} archivos")
        else:
            print(f"{split}/{subdir} no existe.")

Verificando estructura del dataset:
train/images: 1760 archivos
train/labels: 160 archivos
valid/images: 495 archivos
valid/labels: 45 archivos
test/images: 253 archivos
test/labels: 23 archivos


In [27]:
#!zip -r files_backup.zip /content/


Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: content/yolov11/test/images/19_jpg_0_3363.jpg (deflated 0%)
  adding: content/yolov11/test/images/11_jpg_0_1642.jpg (deflated 1%)
  adding: content/yolov11/test/images/11_jpg_0_88.jpg (deflated 3%)
  adding: content/yolov11/test/images/8_jpg_0_8939.jpg (deflated 2%)
  adding: content/yolov11/test/images/19_jpg_0_9251.jpg (deflated 0%)
  adding: content/yolov11/test/images/24_jpg_0_4390.jpg (deflated 2%)
  adding: content/yolov11/test/images/19_jpg_0_5803.jpg (deflated 5%)
  adding: content/yolov11/test/images/Plastico-7_jpg_0_3400.jpg (deflated 0%)
  adding: content/yolov11/test/images/1_jpg_0_5317.jpg (deflated 0%)
  adding: content/yolov11/test/images/Reciclaje-1_jpg_0_6336.jpg (deflated 0%)
  adding: content/yolov11/test/images/Aluminio_jpg_0_6553.jpg (deflated 5%)
  adding: content/yolov11/test/images/19_jpg_0_3507.jpg (deflated 1%)
  adding: content/yolov11/test/images/42_jpg_0_5880.jpg (deflated 5%)
  adding: 

In [28]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [64]:
!git config --global user.name "juanluis77"
!git config --global user.email "juanluiszaratefernandez@gmail.com"


In [65]:
!git config --global init.defaultBranch main

In [67]:
!git init
!git remote add origin https://github.com/juanluis77/entrenamiento_modelo.git
!git add .
!git commit -m "Primer backup desde Colabhhh1"
!git push origin main

Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
[main d8a4405] Primer backup desde Colabhhh1
 1 file changed, 1 insertion(+), 1 deletion(-)
fatal: could not read Username for 'https://github.com': No such device or address


In [68]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [66]:
!git branch


* main


In [63]:
!git branch -M main
